In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(image):
# NOTE: The output you return should be a color image (3 channel) for processing video below
# TODO: put your pipeline here,
# you should return the final output (image with lines are drawn on lanes)

# Read in and grayscale the image
    #image = cv2.imread('/Users/bquinn/Desktop/CarND-LaneLines-P1-master/test_images/solidWhiteRight.jpg') #original
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Next I'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time I am defining a four sided polygon to mask. I realize a limitation will be within the size of the mask which will have to change per video/image frame
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 310), (450, 310), (imshape[1],imshape[0])]], dtype=np.int32)  
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 70     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments

    line_image = np.zeros_like(gray) # I'm creating a blank to draw lines on

    # Run Hough on edge detected image
    # The output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)

    # Here I iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)

    # I'm creating a "color" binary image to combine with line image
    additional_channels = np.dstack((np.zeros_like(line_image),)*(imshape[2]-1))
    multi_channel_line_image = np.dstack((additional_channels, line_image))

    # I'm then drawing the lines on the edge image
    lines_edges = cv2.addWeighted(multi_channel_line_image, 0.8, image, 1, 0)
    #result = cv2.cvtColor(lines_edges, cv2.COLOR_BGR2RGB) #result was line_edges
    #plt.imshow(lines_edges)

    return lines_edges
#I then run my pipline or process_image function on the video. The fl_image functions breaks down the video in individual image frames and applies my pipline.    
white_output = ('/Users/bquinn/Desktop/CarND-LaneLines-P1-master/challenge-after.mp4')
clip1 = VideoFileClip("/Users/bquinn/Desktop/CarND-LaneLines-P1-master/challenge.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)




[[   0  720]
 [ 563  446]
 [ 704  446]
 [1280  720]]


error: /Users/jenkins/miniconda/1/x64/conda-bld/work/opencv-3.1.0/modules/imgproc/src/drawing.cpp:2276: error: (-215) p.checkVector(2, CV_32S) >= 0 in function fillPoly
